In [1]:
pip install fyers-apiv3


/home/vedant/code/FINSTREET-Submission-Signal-Cartel-/activate/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


### **SCRIPT TO GENERATE auth URL**


In [2]:
from fyers_apiv3 import fyersModel
from config import CLIENT_ID, SECRET_KEY, REDIRECT_URI

# Initialize Session Model using credentials from config
session = fyersModel.SessionModel(
    client_id=CLIENT_ID,
    redirect_uri=REDIRECT_URI,
    response_type="code",
    state="hackathon",
    secret_key=SECRET_KEY,
    grant_type="authorization_code",
)

# Generate Auth URL
auth_url = session.generate_authcode()
print("Open this URL in browser:\n", auth_url)

⚠️ WARNING: Access Token file not found. Run the Notebook first!
Open this URL in browser:
 https://api-t1.fyers.in/api/v3/generate-authcode?client_id=6LV74LVY1F-100&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fredirect&response_type=code&state=hackathon


### **GENERATE ACCESS TOKEN**


In [3]:
from fyers_apiv3 import fyersModel
from config import CLIENT_ID, SECRET_KEY, REDIRECT_URI

# 1. PASTE AUTH CODE HERE (From Browser)
AUTH_CODE = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiI2TFY3NExWWTFGIiwidXVpZCI6ImMzOTMwNWIwZDliYTRjYmQ4YzRiNGY1NzcxNWJjNjEzIiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IkZBSDgzOTYzIiwib21zIjoiSzEiLCJoc21fa2V5IjoiYTE4NGIxNjM2NTk3MzRkNTU2MTE1NzI5ZTE2MTFkYTMyOTgyZTM1NzAwZWZjYTZlMDBmY2VkYmIiLCJpc0RkcGlFbmFibGVkIjoiTiIsImlzTXRmRW5hYmxlZCI6Ik4iLCJhdWQiOiJbXCJkOjFcIixcImQ6MlwiLFwieDowXCIsXCJ4OjFcIixcIng6MlwiXSIsImV4cCI6MTc2ODM0NTIxOCwiaWF0IjoxNzY4MzE1MjE4LCJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJuYmYiOjE3NjgzMTUyMTgsInN1YiI6ImF1dGhfY29kZSJ9.k2q-Bc2vskWH5GwiRdvygkGDEu_QSQ4GqdNHx8FWoF8"

# 2. Initialize Session
session = fyersModel.SessionModel(
    client_id=CLIENT_ID,
    secret_key=SECRET_KEY,
    redirect_uri=REDIRECT_URI,
    response_type="code",
    grant_type="authorization_code",
)

# 3. Generate & SAVE Token
session.set_token(AUTH_CODE)
response = session.generate_token()

if response["s"] == "ok":
    access_token = response["access_token"]

    # --- SAVE TO FILE ---
    with open("access_token.txt", "w") as f:
        f.write(access_token)
    print("✅ Success! Access Token saved to 'access_token.txt'")
else:
    print("❌ Error generating token:", response)

✅ Success! Access Token saved to 'access_token.txt'


### **Freeze authentication**


In [4]:
from fyers_apiv3 import fyersModel
from config import CLIENT_ID

# 1. LOAD TOKEN FROM FILE
try:
    with open("access_token.txt", "r") as f:
        access_token = f.read().strip()
    print("✅ Loaded Access Token from file.")
except FileNotFoundError:
    print(
        "❌ Error: 'access_token.txt' not found. Please run the generation step first."
    )


# 2. Initialize Fyers Function
def get_fyers():
    return fyersModel.FyersModel(
        client_id=CLIENT_ID, token=access_token, log_path="./logs"
    )


# Test it
fyers = get_fyers()
print("Fyers Model Initialized!")

✅ Loaded Access Token from file.
Fyers Model Initialized!


### **HISTORICAL DATA FETCH**


In [5]:
from datetime import datetime
import os

# Ensure logs directory exists (FYERS SDK requirement)
os.makedirs("./logs", exist_ok=True)

# Initialize FYERS client
fyers = get_fyers()


def fetch_ohlcv(fyers, symbol, start, end):
    "Fetch daily OHLCV data from FYERS (2-month window)."

    data = {
        "symbol": f"NSE:{symbol}",
        "resolution": "D",  # Daily data only
        "date_format": "0",  # Unix timestamps
        "range_from": int(start.timestamp()),
        "range_to": int(end.timestamp()),
        "cont_flag": "1",
    }

    response = fyers.history(data)
    print(response)  # Keep this for debugging & transparency

    return response

### **HISTORICAL DATA FETCH SANITY CHECK**


In [6]:
start_date = datetime(2025, 11, 1)
end_date = datetime(2025, 12, 31)

resp = fetch_ohlcv(fyers=fyers, symbol="SONATSOFTW-EQ", start=start_date, end=end_date)

{'candles': [[1762128000, 370.25, 374.5, 369.05, 371.7, 266050], [1762214400, 371.7, 376, 368.4, 374.25, 296116], [1762387200, 372.5, 375, 363.1, 365.85, 323191], [1762473600, 363, 369.5, 358, 367.8, 301377], [1762732800, 367.75, 379, 365.1, 376.15, 728662], [1762819200, 376.2, 378, 370.15, 371.8, 299607], [1762905600, 373.8, 396.5, 372, 392.8, 2801504], [1762992000, 393.05, 400.5, 386, 389.4, 995358], [1763078400, 390, 390, 364, 374.15, 2305036], [1763337600, 374, 375, 368.05, 370.95, 874111], [1763424000, 370.1, 370.95, 363.6, 365.5, 623100], [1763510400, 366.5, 376.9, 362.6, 370.05, 1382029], [1763596800, 371.05, 372.45, 362.1, 364.95, 635123], [1763683200, 364.95, 365.9, 356.25, 358.05, 375039], [1763942400, 358.25, 364.5, 358, 360.3, 650417], [1764028800, 362, 364, 355.95, 357.95, 345435], [1764115200, 360.45, 361.45, 356.1, 360.4, 375441], [1764201600, 362.85, 363.05, 355.55, 357.05, 302290], [1764288000, 357.95, 365, 354.4, 357.4, 774003], [1764547200, 358.55, 364.2, 357.2, 358.

### **CONVERTING CANDLES To Pandas Df and Saving to CSV**


In [7]:
import pandas as pd


def candles_to_df(candles):
    df = pd.DataFrame(
        candles, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df.set_index("timestamp", inplace=True)
    return df


df = candles_to_df(resp["candles"])
df.head()

,open,high,low,close,volume
timestamp,,,,,
2025-11-03,370.25,374.5,369.05,371.70,266050
2025-11-04,371.70,376.0,368.40,374.25,296116
2025-11-06,372.50,375.0,363.10,365.85,323191
2025-11-07,363.00,369.5,358.00,367.80,301377
2025-11-10,367.75,379.0,365.10,376.15,728662


In [8]:
df.to_csv("NSE:SONATSOFTW-EQ.csv")
# Saving to CSV